<a href="https://colab.research.google.com/github/NaHyeonMaeng/CODE_Practice/blob/main/BERT_%EC%9E%84%EB%B2%A0%EB%94%A9_%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0_%EB%AA%A8%EB%93%A0_%EC%9D%B8%EC%BD%94%EB%8D%94_%EB%A0%88%EC%9D%B4%EC%96%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

임베딩 추출

In [ ]:
!pip install transformers==2.1.1

In [ ]:
#필요한 모듈 가져오기
from transformers import BertModel, BertTokenizer
import torch

In [ ]:
#사전 학습된 BERT 모델 및 토크나이저 다운로드
#output_hidden_states = True로 설정 -> 모든 인코더 레이어에서 임베딩을 얻는 데 필요함
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 1270526.25B/s]


입력 전처리

In [ ]:
sentence = 'I love Paris'
tokens = tokenizer.tokenize(sentence)
tokens = ['[CLS]'] + tokens + ['[SEP]']

In [ ]:
#PAD 토큰 추가, 어텐션 마스크 정의
tokens = tokens + ['[PAD]'] + ['[PAD]']
attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]

In [ ]:
#토큰을 토큰ID로 변경
token_ids = tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
#텐서로 변환
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

임베딩 가져오기

In [ ]:
#last_hidden_state: 최종 인코더 계층에서만 얻은 모든 토큰의 표현을 가짐
#pooler_output: 최종 인코더 계층의 [CLS] 토큰 표현을 나타내며 선형 및 tanh 활성화 함수에 의해 계산됨
#hidden_states: 모든 인코더 계층에서 얻은 모든 토큰의 표현을 포함
last_hidden_state, pooler_output, hidden_states = model(token_ids, attention_mask = attention_mask)

In [ ]:
#[batch_size, sequence_length, hidden_size]
last_hidden_state.shape

torch.Size([1, 7, 768])

In [ ]:
#[batch_size, hidden_size]
pooler_output.shape

torch.Size([1, 768])

In [ ]:
#입력 임베딩 레이어 h0에서 최종 인코더 레이어 h12까지 모든 인코더 레이어의 표현을 포함하는 13개의 값을 포함하는 튜플
len(hidden_states)

13

In [ ]:
#입력 임베딩 레이어 h0에서 얻은 모든 토큰의 표현 벡터를 가짐
#[batch_size, sequence_length, hidden_size]
hidden_states[0].shape

torch.Size([1, 7, 768])